In [1]:
import re
import requests
import pandas as pd
from Bio import SeqIO
from Bio.KEGG import REST
from Bio.KEGG.KGML import KGML_parser

In [2]:
all_reactions = pd.read_csv("reactionpresence.csv", nrows=1).columns.tolist()
all_reactions_strip = [x[:-2] for x in all_reactions]
all_reaction_dict = dict.fromkeys(all_reactions_strip)

In [3]:
r = requests.get("https://raw.githubusercontent.com/ModelSEED/ModelSEEDDatabase/master/Biochemistry/reactions.tsv")
r = r.content.decode("utf-8")

In [4]:
for key in all_reaction_dict:
    annotations = r
    present = annotations.find(key)
    if present != -1:
        right = annotations[present:]
        position = right.find("KEGG: ")
        if position != -1:
            all_reaction_dict[key] = right[position+6:position+13]
        elif position == -1:
            all_reaction_dict[key] = "NA"
    elif present == -1:
        all_reaction_dict[key] = "NA"
clean_dict = {key.strip("|"): item.strip("|") for key, item in all_reaction_dict.items()}

In [5]:
for key, value in clean_dict.items():
    try:
        kegg = REST.kegg_get(value).read()
        position = kegg.find("Metabolism; ")
        if position != -1:
            clean_dict[key] = re.search('Metabolism;(.*)\n', kegg)
        elif position == -1:
            clean_dict[key] = "NA"
    except:
        pass

In [6]:
with open('reaction_annotations.csv', 'w') as f:
    for key in clean_dict.keys():
        f.write("%s,%s\n"%(key,clean_dict[key]))